In [2]:
import pandas as pd
import numpy as np
import random
import math
import csv
raw_data = pd.read_csv('train-v2.tsv', sep="\t", header=None, quoting = csv.QUOTE_NONE)
raw_data.columns = ['label', 'tweet']
raw_data = raw_data[pd.notnull(raw_data['tweet'])]
raw_data.head()

,label,tweet
0,0,@USER @USER a sicrhau bod mwy o arian poced 'd...
1,1,Parti Dolig da gyda tim swyddfa canolog @USER ...
2,0,@USER yeaah ma fe yn wir. ( oh well.
3,1,@USER hahaha idk. 3am oedd y bws ti?
4,0,@USER dwim yn gal llun ohoni?


In [3]:
print(raw_data.shape)
raw_data.head(100)

(80000, 2)


,label,tweet
0,0,@USER @USER a sicrhau bod mwy o arian poced 'd...
1,1,Parti Dolig da gyda tim swyddfa canolog @USER ...
2,0,@USER yeaah ma fe yn wir. ( oh well.
3,1,@USER hahaha idk. 3am oedd y bws ti?
4,0,@USER dwim yn gal llun ohoni?
...,...,...
95,1,Mae ein ceidwad Jim wedi cymryd lluniau hyfryd...
96,0,Amserlen @USER a'r plant heddiw:@USER (sesiwn ...
97,1,@USER dim problem. Diolch am y gusan Aled
98,1,@USER Ffab. Gweld ti prawn Dydd Sul y 20th de ...


In [4]:
# Appears to be an almost perfectly balanced dataset.  Our "priors" are 50/50
print(raw_data[raw_data['label']==0].shape[0])
print(raw_data[raw_data['label']==1].shape[0])

39981
40019


In [5]:
# Hold back training data
training_data = raw_data.sample(frac=0.8)
testing_data = raw_data.drop(training_data.index)

In [6]:
print(training_data.shape)
print(testing_data.shape)

(64000, 2)
(16000, 2)


In [7]:
# Separate training data into positive and negative datasets for simplicity
training_data_pos = training_data[training_data['label']==1]
training_data_neg = training_data[training_data['label']==0]

In [8]:
print(training_data_neg.shape)
print(training_data_pos.shape)

(32010, 2)
(31990, 2)


In [9]:
training_data_neg.head

<bound method NDFrame.head of        label                                              tweet
70445      0  @USER you cutie! deffos coming to visit cyn dy...
22728      0  @USER yo! t nol yn liverpool eto gai? ta t ddi...
48599      0                 @USER @USER a fi ar ol ysgol tan 5
63052      0  @USER @USER @USER fyddna phonecall gan y polic...
57810      0  @USER dim yn edrych yn well i'r ddau mynudd ne...
...      ...                                                ...
28436      0  @USER nath o jest hito fi bo ni'n 'tyfu fyny'....
71648      0        Methu gwitchad i fynd adra i miss my famlam
50620      0                       @USER dylsa chdi di dod beth
15295      0               Trist heno meddwl am bawb yn Boston.
72996      0  @USER Haha dwi'n gwbod! 😳 Mae'n iawn, bai @USE...

[32010 rows x 2 columns]>

In [10]:
training_data_pos.head

<bound method NDFrame.head of        label                                              tweet
51508      1  @USER m8 dim ond un dysgesi llynedd a we hwna ...
50768      1  Wedi joio #nowe yn @USER cerwch os gewch chi g...
26240      1    @USER @USER dim mewn llyfr dwim yn malio dim am
65391      1  Massive Penblwydd Hapus I @USER. Finally 18! G...
72530      1  @USER Diolch btw pwy ydi Chandian pan mae o ad...
...      ...                                                ...
39604      1  @USER he heh, prif reswm imi ddechre tumblo yd...
38692      1                                  @USER ar dan heno
27291      1  @USER dim syniad, falle! siwr bod nhwn cymyd l...
68800      1       @USER yndi . Sut mae'r papur yn mynd? :') xx
61596      1  @USER @USER Pob lwc!!! Bechod fod batri dy hen...

[31990 rows x 2 columns]>

In [11]:
# We need to calculate/generate the following:
# Number of unique words in positive tweets
# Number of unique words in negative tweets

# We could make a dictionary of unique words as keys, values as number of times the word appears

In [34]:
uniquePositiveWords={}
uniqueNegativeWords={}
uniqueTotalWords={}

In [36]:
for tweet in range(len(training_data_pos)):
    currentTweet=training_data_pos.iloc[tweet]['tweet'].split()
    for word in currentTweet:
        if word in uniquePositiveWords:
            uniquePositiveWords[word]+=1
        else:
            uniquePositiveWords[word]=1
for tweet in range(len(training_data_neg)):
    currentTweet=training_data_neg.iloc[tweet]['tweet'].split()
    for word in currentTweet:
        if word in uniqueNegativeWords:
            uniqueNegativeWords[word]+=1
        else:
            uniqueNegativeWords[word]=1
for tweet in range(len(training_data)):
    currentTweet=training_data.iloc[tweet]['tweet'].split()
    for word in currentTweet:
        if word in uniqueTotalWords:
            uniqueTotalWords[word]+=1
        else:
            uniqueTotalWords[word]=1

In [37]:
numUniquePositiveWords=len(uniquePositiveWords)
numUniqueNegativeWords=len(uniqueNegativeWords)
numUniqueTotalWords=len(uniqueTotalWords)
print(numUniquePositiveWords)
print(numUniqueNegativeWords)
print(numUniqueTotalWords)


69388
66878
112384


In [39]:
uniquePositiveWords

{'@USER': 68098,
 'm8': 16,
 'dim': 2116,
 'ond': 3476,
 'un': 2344,
 'dysgesi': 2,
 'llynedd': 20,
 'a': 12826,
 'we': 228,
 'hwna': 256,
 'loads': 116,
 'o': 12400,
 'effort.': 6,
 'Wedi': 692,
 'joio': 428,
 '#nowe': 4,
 'yn': 35098,
 'cerwch': 6,
 'os': 2056,
 'gewch': 56,
 'chi': 2256,
 'gyfle.': 2,
 "Effin'": 2,
 'disgraceful': 2,
 '😳': 30,
 'mewn': 1690,
 'llyfr': 144,
 'dwim': 368,
 'malio': 2,
 'am': 9078,
 'Massive': 6,
 'Penblwydd': 858,
 'Hapus': 686,
 'I': 3130,
 '@USER.': 224,
 'Finally': 12,
 '18!': 6,
 'Gobeitho': 30,
 'wenes': 2,
 'ti': 6000,
 'neithwr': 30,
 'joia': 132,
 'rest': 20,
 'or': 420,
 'diwrnod': 586,
 '#jones': 2,
 'xx': 2478,
 'Diolch': 2718,
 'btw': 120,
 'pwy': 364,
 'ydi': 252,
 'Chandian': 2,
 'pan': 926,
 'mae': 1548,
 'adra?!': 6,
 'Gen': 20,
 'i': 20888,
 'rai': 50,
 'gwaeth': 24,
 'explainai': 4,
 'fori': 92,
 'gdgd': 10,
 'ye': 86,
 'went': 26,
 'good': 776,
 'even': 144,
 'though': 152,
 'odd': 1242,
 '2': 584,
 'bloke': 2,
 'ifanc': 86,
 'na': 

In [41]:
# Calculate word sentiment scores
for key in uniquePositiveWords:
    uniquePositiveWords[key]=math.log((uniquePositiveWords[key]+1)/(numUniquePositiveWords+numUniqueTotalWords))
for key in uniqueNegativeWords:
    uniqueNegativeWords[key]=math.log((uniqueNegativeWords[key]+1)/(numUniqueNegativeWords+numUniqueTotalWords))

In [42]:
uniquePositiveWords

{'@USER': -0.981790625720409,
 'm8': -9.27729508940617,
 'dim': -4.45275316232752,
 'ond': -3.956583301454524,
 'un': -4.350467752582006,
 'dysgesi': -11.011896144794276,
 'llynedd': -9.065985995738963,
 'a': -2.651200830162535,
 'we': -6.676786429908145,
 'hwna': -6.561432348567165,
 'loads': -7.348334498664629,
 'o': -2.684976039959613,
 'effort.': -10.164598284407072,
 'Wedi': -5.569478434272482,
 'joio': -6.049051514534368,
 '#nowe': -10.501070521028286,
 'yn': -1.6445805144427668,
 'cerwch': -10.164598284407072,
 'os': -4.481504543809428,
 'gewch': -8.067457165627836,
 'chi': -4.38871665664485,
 'gyfle.': -11.011896144794276,
 "Effin'": -11.011896144794276,
 'disgraceful': -11.011896144794276,
 '😳': -8.67652122897724,
 'mewn': -4.6774330845638055,
 'llyfr': -7.133774691041811,
 'dwim': -6.199711789421858,
 'malio': -11.011896144794276,
 'am': -2.9967891000906284,
 'Massive': -10.164598284407072,
 'Penblwydd': -5.35473951147813,
 'Hapus': -5.578174141240036,
 'I': -4.06140071213597

In [43]:
uniqueNegativeWords

{'@USER': -1.247441387729363,
 'you': -5.78486889267489,
 'cutie!': -10.487165789393703,
 'deffos': -9.698708429029434,
 'coming': -9.531654344366267,
 'to': -5.459345670543347,
 'visit': -9.263390357771588,
 'cyn': -5.266809964315379,
 'dydd': -4.375698449891026,
 'gwener!': -9.698708429029434,
 'ffili': -7.092657395882345,
 'credu': -6.276520771475442,
 'bod': -3.542692927476404,
 'ysgol': -5.320096709455621,
 'wedi': -4.106026820083882,
 'benu': -9.152164722661364,
 '#emotionalisnottheword': -10.997991413159694,
 'yo!': -10.487165789393703,
 't': -6.637018187683645,
 'nol': -4.71946998899385,
 'yn': -1.5252098565975258,
 'liverpool': -9.263390357771588,
 'eto': -5.336189010744376,
 'gai?': -10.997991413159694,
 'ta': -6.444114521559154,
 'ddim': -3.3083152415641846,
 'am': -3.106037563696227,
 'ddod': -5.639834046255641,
 'nol?': -8.600096140361323,
 'dwi': -3.440866700963508,
 'neu': -5.1486666332128355,
 'drawings': -10.997991413159694,
 'v': -6.196706348245312,
 'heddiw': -4.7577

In [44]:
testing_data

0        @USER @USER a sicrhau bod mwy o arian poced 'd...
7        @USER mar boy yn hymian o nappies. Dwim isho s...
10       @USER ond paid a meiddio darllen y tweet yma a...
13       @USER hm dwi'r un peth! genai cur yn pen 24/7 ...
16         really methu symud mas o gwely #socomfy #gwaith
                               ...                        
79976    @USER Ti wnaeth dangos y ffordd i mi yn y lle ...
79984    Ffonio mam i warnio'i bo fi di colli nail varn...
79986             @USER pam del? mewn mood ti? bechs;o oke
79994                        Newyddion ffug 😉 #yagym {URL}
79998    @USER eidiaaaal, awni dydd llun nesa ia? Y air...
Name: tweet, Length: 16000, dtype: object

In [45]:
testing_data=testing_data['tweet']

KeyError: 'tweet'

In [ ]:
testing_data

In [22]:
for tweet in testing_data:
    currentScore=[]
    for word in tweet:
        if word in uniquePositiveWords:
            currentScore.append(uniquePositiveWords[word])
    #print(currentScore)
    tweetScore = np.prod(currentScore)
    print(tweet, tweetScore)

        
        

@USER @USER a sicrhau bod mwy o arian poced 'da ti, nes i fe i ti, Efa, jst i ti never again, creulondeb i ddyffryn Aman! 5.13177516550216e+74
@USER mar boy yn hymian o nappies. Dwim isho siarad efo do os dos nam rhaid. A dwnim faint o gloch dwi fod i gyrradd crewe -1.267854866976148e+74
@USER ond paid a meiddio darllen y tweet yma allan ar y rhaglen de #cringe 3.7706676870023994e+46
@USER hm dwi'r un peth! genai cur yn pen 24/7 fyd duww:0 combination horrible! pop lwc! genai cymraeg dydd gwener a tgch wsos nesa xx -5.161324605152855e+88
really methu symud mas o gwely #socomfy #gwaith 6.761167466909963e+32
@USER mana riwyn di dwyn pres fi a bob dim, dwim yn gallu neud dim :/ sorri boi 1.6932922831380356e+48
Happy New Year all! Blwyddyn Newydd Dda! 1.4331055598598848e+27
Back to reality. Gwaith cyn parti Hairspray heno -5.353780046515862e+30
Ffiseg done...gorffan geog...attemptio neu cinio a hello llafar cymraeg -1.5940627563448257e+49
@USER chos doni'm yn gwbo bod chdi off sani di gall

Off i weld fy mhwyllgor cynta ac yn edrych mlan at ymweld yr Uned Iaith. Diwrnod ola! -1.1648273306540005e+54
@USER exactly! Ia, yn diwadd mashwr nai brynu fo jysd ella nai witchad tan rol holiday fi i fi allu fforddio fo 1.804419760554701e+68
@USER cwrw wrth gwrs! ond rwyn hoffi llaeth hefyd -3.3035526172550624e+35
@USER Hefyd o'n i'n rili edrych 'mlaen i weld dy lincs di o gwmpas fy eitem 'Chwalu' yn y gyfres newydd x #gutted -4.74970439626577e+73
@USER just gadewch i'ch music chi siarad (nes i fwynhau lot gyda llaw!) -5.1625731949602e+48
Mae'r lolfa ar gau officially #tramps 3.5013006474171187e+25
Cael imbud ond trafferth fo iphone 6 fi 2.999945043521544e+25
oooch marw sho nightout yn bangor heno 3.2202035378337043e+24
@USER gytud. ♡'r gân. Playlist i ar goll hebddi -3.4021584838959664e+32
Siriys amser cysgu wan neu fyddai 'tha bechdan 'fory !!! #clebran #angen #cwsg 1.7776263644396522e+53
@USER @USER hahah a gal fy contio bora dydd llun wedyn mashwr 1.4242930653454538e+41
@USER @US

@USER Diolch yn fawr am y neges neis! Byddwn ni'n ychwanegu cardiau drwy'r amser - wastad rhywbeth i'w ddathlu! -7.444221571709536e+74
@USER @USER Fi hefyd ar gyfer yr 2il hanner. Llun lot yn well... 3.3445694856553063e+43
heb weld NEB drw dydd heddiw #llemapawb ? 2.08129527576287e+29
@USER aww diolch catrin! Nawr fi'n gweld hwn sori! Gobeithio nes ti joio! -7.882265635411448e+47
@USER wna i, diolch 13204004226475.262
@USER Reit dda - gan mod i angen mynd ar y 'school run' syth wedyn !! 4.450457782723486e+42
3 diwrnod yn unig ar ôl i archebu a'u derbyn cyn y Nadolig {URL} -1.3551687081636616e+38
@USER yn dechnegol - mae nhw di curo'r Faeroes mewn gêm gyfeillgar mis diwetha :o 1.5435302759127204e+54
@USER Hi Tudur. Wedi ebostio'r llynia draw i chdi gael eu gweld. Joia... 2.6011623927967236e+47
@USER yndw..ahaha! Di bod trw 5 heno, fyddai ar ddeffro tans riw 3-4 heno wan -1.0020565803510743e+49
Ma'r llun yn ddigon! No words needed! #Nutella #Nutella&Go #Wednesdaytreat #chocolate #HappyTr

Newydd edrych ar arlwy ddyddiol #S4C- Wedi'i theilwra yn bennaf i fabanod a phlant.. Be am y rheiny nad sydd am wylio Cyw, 9 awr y dydd! 2.9409178287263657e+83
@USER gobeithio😭😘 fedrai'm siarad efo pobl ddo SAVE ME 3.292723538080652e+37
dwin nacyd erbyn riw 2 dyddia' ma', motch faint dwi'n cysgu #yaaaaawn 1.3147509966254482e+43
Ffordd ar gau yng nghanol Caerdydd wrth i waith adeiladu ddechrau >>{URL} {URL} -1.1052014930787078e+53
@USER Oh na. Ella chdi ydi'r jinx felly !! 12 hour shift bob game i chdi wan 😉 -3.760309662454539e+49
@USER @USER y am I so ugly y -7.792632966034449e+17
@USER @USER Nath Sgwbi alw fi'n dew wedyn 'rol ni weld chi haha diolch, a chditha! -3.516790566894308e+55
@USER aye! Dedicated graphics, i5 a 8gb ram muahaha. 1tb hdd fyd -6.815426395480764e+42
@USER W, le ges t linen trousers navy? V di bo n wilo am sbelen i gwaith. Bygger all..car yn y garej nes d.llun x -1.0889112686743043e+72
di cal noson dda yn parti saran wooooo! #fun 5.609448442916396e+25
@USER ooo oed

.@USER Hey Tuds - Fedra chdi Signio yr Online Petition am Sefyllfa fi efo'r Cyngor {URL} - Diolch i ti mate 7.177846084784937e+67
@USER bydd rhai o’ch dilynwyr yn derbyn eu canlyniadau Lefel A ar 14/08 yn gweld y fideo yma yn ddefnyddiol URL} AD -3.7989253466721356e+72
@USER @USER Diolch am y croeso ar y rhaglen. Joies i. Gobeitho nath pawb nath wylio joio #LlywyddyrUrdd 6.835139793422124e+67
@USER rho llunie sy ar ipod ti lan os ti galle xxx -2.834444977148944e+30
“@USER: Teyrnged i'r Dandy ar BBC Scotland rwan. Gorffwys mewn hedd yr hen gomic annwyl ” amen i hynny -3.370086957664944e+66
Canlyniad siomedig ond diolch @USER @USER @USER a @USER am y sylwebaeth 3.6329358174693794e+49
@USER Diolch,dwi mewn panic mode rwan haha,nai siarad efo chi wedyn <3 x 5.72233929643706e+46
@USER a ddim y ffrindia gora os nad wyt ti 100% #gwaethnalloerig 3.7825438740950666e+40
@USER Bonjour! Sori rwan dwi'n gweld y neges, diolch yn fawr teimlo'n well rwan, ti'n ok? 4.778538549866951e+59
dwin rili licio

...

@USER hahaha ddim y back of beyond dim mwy 8.17806131306663e+26
@USER ges di rai? Fi ffion a haw drio bob date a methu 4.5183985467507775e+32
@USER Diolch am dod i Llangefni heddiw Scott #littlestars #rugby #wales #goglove {URL} -1.9058392341306222e+61
Pam wythnos mor wych ac yna ffeindio un peth bach allan sy'n dod a'r cwbl i'r llawr mewn eiliad? 7.435624295991528e+61
@USER Mae fy gwrando a darllen yn wan o hyd. -2.577306236622109e+27
dylsa fi ddim di mynd i gym cyn netball mewn poen !! -4.242858484833906e+31
methu ffindio'r mynadd i stydio 📝 -1.61985705356008e+20
@USER Paid â bwyta gormod ohonyn nhw neu #MidriffCrisis fydd gyda ti! 2.102325308279482e+45
@USER Na nid diflads ond calonogol i eraill ar y daith 2.976297427542814e+33
@USER @USER @USER odyn ni'n mynd owh, gewn ni drink dros chi xxxx -2.343133926056034e+43
@USER na dwim yn mel achos gwres a dolur gwddw su arnai 1.8711457202387043e+36
Mae ein "tag" (o'r diwedd!) wedi newid i @USER ar gyfer y ffrwd Cymraeg a @USER ar gyfer y 

@USER garantid bo chdi sa nw di gallu arrangio wbath I chi sa -3.487909693992964e+38
Devastated bofi di tori screen iphone fi eto.. Ail waith wan 🙍 2.4392300497039785e+37
Fedrai'm coelio mewn 15 diwrnod fyddai di gorffen fy arholiadau a fy mlwyddyn gynta... Gai fynd nôl i wsos freshers plis -3.3750786418443027e+76
@USER ofi'n edrych lawr grisha am ages cyn sylwi odd y llyfra dwisho fyny grisha haha! -3.179107858703566e+56
@USER @USER @USER Pawb yn credu mewn annibyniaeth. Gallwn drafod y gweddill ar ôl cyrraedd y nod! #diplomat -2.3627806736337335e+72
@USER shame! It was a really good episode!:o llawer o ddrama 5.713264377047546e+39
@USER mae o'n deud ar yr arwydd uwch dy ben di 8.962890304819583e+28
Pen yn brifo bora ma 731955891848.7139
@USER dwi'n sal a di laptop fi ddim yn gweithio a BLEURGH 9.341663255323271e+36
@USER flin rwy'n hwyr ond llongyfarchiadau! 5.210299958085358e+30
Lwmp i annwyd #LovingTheOfficeLife #LivingTheDream 2.051221744807219e+37
@USER naa fyddo'n waeth 😭 -1.107

@USER onim yn sal at all sdi oni wide awake 6am jus sal sal ye!.. oh ia haha cofia amdana fi os tn curo bwbw 1.2662283856344128e+64
@USER Wel weithiau dwi'n anghofio geiriau oherwydd dwi heb gwneud Cymraeg am tua tri blwyddyn. So weithiau mae'n rhaid i mi edrych -5.238734051325313e+84
@USER awh diolch cariad! haha okay we'll see about that, sai'n poser lol! oren yn cerdded rownd dre gine ond ma'n sound nawr 3.981883120143051e+80
Falch i ddeud bo fi wedi studio chydig o maths bora ma, nol cinio i Pant du yn y munud efo elliw a elin a mwy o studio ar ol hyna 📚 1.7023229812173354e+75
@USER - hehe ur tough :P odda nhw'n fuckin lladd i fi de + True ! Dwi efo nhw ers blwyddyn a hannar -_- x -1.7880935113096744e+66
@USER ti'n rhydd heno? -7913163539718288.0
@USER @USER yn iawn helen will be proud ffiffs! haa yee ma'n rhwyddach i ddeall fyd:L hahaha fimod, so funny mind!xxx -1.6679243653990576e+78
@USER @USER lol. Alli di fynd ag unrhwbeth ond dim fy ngwallt!! 5.211508956664193e+43
Isio mynd i

@USER Diolch Catrin. Gobitho ath gweddill y ffilmio yn iawn ddydd mowrth! 5.833772507269026e+48
Er dy fod ymhell i ffwrdd dwin teimlo gwres dy gariad <3 dal heb taro adra bywyd mor greulon.. 2.44886866460779e+60
@USER Same here Mel! Dwi'm di weld o chwaith 3.763874608098479e+30
Exams man draggio wan, studio am wsos arall yn gwbod bo fi am failio nhw 1.7371125634067197e+44
@USER Diolch ti Caiber, os fedrai aros yn ddeffro drwy 90 mins!! Haha 😴 💩👶🏻 1.5830096277003875e+48
ENWEBU PROSIECT yn eich ardal i wella amgylchedd neu wasanaethau? Ffurflen ar-lein URL} #ad-dalucymuned 2.2064594924296441e+74
@USER yndwf tad fyddai'n nol mewn riw hannar awr mashwr xxx -9.461129434274145e+38
@USER argol ti'n faleisus!! Gobeithio di plant heddi sy'n dy ddilyn ddim un copio dy esiampl #mannars -1.2828794370184952e+66
Metastwnsh yn diflannu gyda esboniad sydd yn dweud fod nifer o resymau ond ddim yn dweud beth yw'r rhesymau. Oes yna gagging clause? 2.551063559583205e+86
@USER Fflio ar y 12th o December do

@USER gweithio babs dwi adra 28th March am wsos so fu rhaid ni neud wbath os ti rhydd. Bangor?? Heb fod ers bday Meg😯 -8.077026131293687e+74
@USER French, Welsh a phys i fi! Dim yn meindio pt tbh -4.601666008948027e+35
@USER why thankyou Jodsx gweld ti wythnos nesa 😘😘 ffily aros xx 3.867385947924691e+41
@USER @USER @USER @USER @USER Ar ôl gweld y pennod, mae hwn yn hilariws hehe. Chwarae teg 1.0539324045922486e+63
.@USER Felly, dwi wedi penderfynu dod mas â brand newydd o'r condoms Cymreig gorau - Pwyll! a Pryderi! -1.2248624206661917e+68
@USER @USER Www posh yn ty chi ! -1.0429108721886937e+22
Bored o fod yn unig blentyn @USER angen brysho adra a Gruff angen brysho nol o sbaen !!!!!! 1.2736356777780388e+58
@USER awen yn bonkyrs 476540649527613.3
Gweld fod #Wales #Scott Williams a #triplecrown yn trendio yn Llundain ) -3.806446831790226e+48
Newydd ddod i mewn.Falch bod pawb yn mwynhau cyngerdd yr Urdd ))) 3.165591197557837e+43
@USER di ffon ti'n gwithio bellach? Colli nabod ana chdi he

In [23]:
for tweet in testing_data:
    currentScore=[]
    for word in tweet:
        if word in uniqueNegativeWords:
            currentScore.append(uniqueNegativeWords[word])
    #print(currentScore)
    tweetScore = math.log(np.prod(currentScore))
    print(tweet, tweetScore)

@USER @USER a sicrhau bod mwy o arian poced 'da ti, nes i fe i ti, Efa, jst i ti never again, creulondeb i ddyffryn Aman! 170.36357946367332


ValueError: math domain error